# Comparison of EigenFace and FisherFace + FaceFrontalization

Using KNN as the classifier

In [ ]:
import os

os.chdir("..")

In [ ]:
import cv2
import random
import numpy as np
import matplotlib.pyplot as plt
import urllib.request as urlreq

from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_score, recall_score, accuracy_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

import scipy.io as scio
from face_frontalization import frontalize
from face_frontalization import camera_calibration as calib

%load_ext autoreload
%autoreload 2

In [ ]:
min_image_num = 10
training_set_size = 10
training_image_num = 5
num_misclassified_to_show = 5

frontalize_face = True
crop_face_after_norm = True
face_image_target_size = (64, 64)
base_folder = "data/face/lfw-deepfunneled/lfw-deepfunneled"

haarcascade = "model_checkpoints/haarcascade.xml"
haarcascade_url = "https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_alt2.xml"

LBFmodel = "model_checkpoints/lbfmodel.yaml"
LBFmodel_url = "https://github.com/kurnianggoro/GSOC2017/raw/master/data/lbfmodel.yaml"

frontalize_model_name = "model_dlib"
frontalize_model_path = "model_checkpoints/model3Ddlib.mat"

eye_mask_mat = "eyemask"
eye_mask_mat_path = "model_checkpoints/eyemask.mat"

In [ ]:
for filename, url in zip([haarcascade, LBFmodel], [haarcascade_url, LBFmodel_url]):
    if os.path.exists(filename):
        print("File exists")
    else:
        urlreq.urlretrieve(url, filename)
        print("File downloaded")

In [ ]:
data = {}

for person in os.listdir(base_folder):
    person_dir = os.path.join(base_folder, person)
    if os.path.isdir(person_dir):
        images = []
        for img_name in os.listdir(person_dir):
            img_path = os.path.join(person_dir, img_name)
            img = cv2.imread(img_path)
            if img is not None:
                images.append(img)
        if len(images) >= min_image_num:
            data[person] = images

In [ ]:
x_train = []
y_train = []
x_test = []
y_test = []

people_names = list(data.keys())

for person, images in data.items():
    train_images = []
    test_images = []
    random.shuffle(images)

    train_images.extend(images[:training_image_num])
    test_images.extend(images[training_image_num:])
    train_labels = [person] * training_image_num
    test_labels = [person] * (len(images) - training_image_num)

    other_people = random.sample(
        [p for p in people_names if p != person],
        2 * (training_set_size - training_image_num),
    )

    for i, other_person in enumerate(other_people):
        chosen_image = random.sample(data[other_person], 1)[0]
        if i % 2 == 0:
            train_images.append(chosen_image)
            train_labels.append("Unknown")
        else:
            test_images.append(chosen_image)
            test_labels.append("Unknown")

    x_train.append(train_images)
    x_test.append(test_images)
    y_train.append(train_labels)
    y_test.append(test_labels)

In [ ]:
face_detector = cv2.CascadeClassifier(haarcascade)
landmark_detector = cv2.face.createFacemarkLBF()
landmark_detector.loadModel(LBFmodel)

In [ ]:
def normalize_face(img):
    model3D = frontalize.ThreeD_Model(frontalize_model_path, frontalize_model_name)

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_detector.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
    if len(faces) == 0:
        raise RuntimeError("No faces detected.")

    main_face = np.array([max(faces, key=lambda rect: rect[2] * rect[3])])
    retval, landmarks = landmark_detector.fit(gray, main_face)
    if not retval or len(landmarks) == 0:
        raise RuntimeError("Could not detect landmarks.")

    # OpenCV returns landmarks as a list, where each element is an array of shape (1, 68, 2).
    lmarks = landmarks[0][0]
    proj_matrix, _, _, _ = calib.estimate_camera(model3D, lmarks)

    eyemask = np.asarray(scio.loadmat(eye_mask_mat_path)[eye_mask_mat])
    frontal_raw, frontal_sym = frontalize.frontalize(
        img, proj_matrix, model3D.ref_U, eyemask
    )

    return frontal_raw, frontal_sym


def obtain_only_face(frontal_view):
    faces = face_detector.detectMultiScale(
        frontal_view, scaleFactor=1.1, minNeighbors=5
    )

    main_face = np.array([max(faces, key=lambda rect: rect[2] * rect[3])])
    _, landmarks = landmark_detector.fit(frontal_view, main_face)

    lmarks = landmarks[0][0]
    hull = cv2.convexHull(np.array(lmarks, dtype=np.int32))

    min_x = min(lmarks, key=lambda p: p[0])[0]
    max_x = max(lmarks, key=lambda p: p[0])[0]
    min_y = min(lmarks, key=lambda p: p[1])[1]
    max_y = max(lmarks, key=lambda p: p[1])[1]

    mask = np.zeros((frontal_view.shape[0], frontal_view.shape[1]), dtype=np.uint8)
    cv2.fillPoly(mask, [hull], 255)

    masked_face = frontal_view.copy()
    if masked_face.dtype != np.uint8:
        masked_face = np.uint8(np.clip(masked_face, 0, 255))

    masked_face[mask == 0] = 0
    masked_face = masked_face[
        int(min_y) - 5 : int(max_y) + 5, int(min_x) - 5 : int(max_x) + 5
    ]

    masked_face = cv2.cvtColor(masked_face, cv2.COLOR_BGR2GRAY)
    resized_face = cv2.resize(masked_face, face_image_target_size)
    return resized_face


def normalize_list(image_list, useSym):
    if crop_face_after_norm:
        return [
            obtain_only_face(normalize_face(x)[1 if useSym else 0]) for x in image_list
        ]

    return [normalize_face(x)[1 if useSym else 0] for x in image_list]

In [ ]:
def calculate_metrics(true_labels, predicted_labels):
    precision = precision_score(
        true_labels,
        predicted_labels,
        average="weighted",
        labels=np.unique(true_labels),
        zero_division=0,
    )

    recall = recall_score(
        true_labels,
        predicted_labels,
        average="weighted",
        labels=np.unique(true_labels),
        zero_division=0,
    )

    accuracy = accuracy_score(true_labels, predicted_labels)
    return precision, recall, accuracy


def track_misclassifications(test_images, true_labels, predicted_labels):
    misclassified_images = []
    misclassified_true_labels = []
    misclassified_pred_labels = []

    true_labels = np.array(true_labels)
    misclassified_indices = np.where(predicted_labels != true_labels)[0]

    for idx in misclassified_indices:
        misclassified_images.append(test_images[idx])
        misclassified_true_labels.append(true_labels[idx])
        misclassified_pred_labels.append(predicted_labels[idx])

    return misclassified_images, misclassified_true_labels, misclassified_pred_labels


def visualize_misclassifications(
    title, misclassified_images, misclassified_true_labels, misclassified_pred_labels
):
    misclassified_indices_sample = random.sample(
        range(len(misclassified_images)),
        min(num_misclassified_to_show, len(misclassified_images)),
    )

    plt.figure(figsize=(15, 4))

    for idx, misclassified_idx in enumerate(misclassified_indices_sample):
        image = misclassified_images[misclassified_idx]
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        true_label = misclassified_true_labels[misclassified_idx]
        predicted_label = misclassified_pred_labels[misclassified_idx]

        plt.subplot(1, num_misclassified_to_show, idx + 1)
        plt.imshow(image_rgb)
        plt.title(f"True: {true_label}\nPred: {predicted_label}")
        plt.axis("off")

    title = f"{title} ({f'With Face Frontalization and{" without" if not crop_face_after_norm else ""} Face Croping' if frontalize_face else 'Without Face Frontalization'})"
    plt.suptitle(title, fontweight="bold")
    plt.tight_layout()
    plt.show()


def print_metrics(title, all_precision, all_recall, all_accuracy):
    average_precision = np.mean(all_precision)
    average_recall = np.mean(all_recall)
    average_accuracy = np.mean(all_accuracy)

    title = f"{title} ({f'With Face Frontalization and{" without" if not crop_face_after_norm else ""} Face Croping' if frontalize_face else 'Without Face Frontalization'})"

    print(title)
    print(f"Average Precision: {average_precision}")
    print(f"Average Recall: {average_recall}")
    print(f"Average Accuracy: {average_accuracy}")

In [ ]:
def eval_fisherface(useSym):
    all_precision = []
    all_recall = []
    all_accuracy = []

    misclassified_images = []
    misclassified_true_labels = []
    misclassified_pred_labels = []

    for i in range(len(data)):
        try:
            training_label_set_person_i = y_train[i]
            training_set_person_i = (
                normalize_list(x_train[i], useSym) if frontalize_face else x_train[i]
            )

            testing_label_set_person_i = y_test[i]
            testing_set_person_i = (
                normalize_list(x_test[i], useSym) if frontalize_face else x_test[i]
            )

            train_images_flat = np.array([p.flatten() for p in training_set_person_i])
            test_images_flat = np.array([p.flatten() for p in testing_set_person_i])

            # Avoid singularity issue in LDA and help generalize
            pca = PCA(n_components=min(5, train_images_flat.shape[0] - 1))
            train_pca = pca.fit_transform(train_images_flat)
            test_pca = pca.transform(test_images_flat)

            lda = LDA()
            lda.fit(train_pca, training_label_set_person_i)

            train_fisherfaces = lda.transform(train_pca)
            test_fisherfaces = lda.transform(test_pca)

            knn = KNeighborsClassifier(n_neighbors=1)
            knn.fit(train_fisherfaces, training_label_set_person_i)
            test_predictions = knn.predict(test_fisherfaces)

            precision, recall, accuracy = calculate_metrics(
                testing_label_set_person_i, test_predictions
            )

            all_precision.append(precision)
            all_recall.append(recall)
            all_accuracy.append(accuracy)

            (
                misclassified_batch_images,
                misclassified_batch_true_labels,
                misclassified_batch_pred_labels,
            ) = track_misclassifications(
                testing_set_person_i,
                testing_label_set_person_i,
                test_predictions,
            )

            misclassified_images.extend(misclassified_batch_images)
            misclassified_true_labels.extend(misclassified_batch_true_labels)
            misclassified_pred_labels.extend(misclassified_batch_pred_labels)
        except Exception as e:
            print(
                f"An error occurred at index {i} for one of the images: {e} Skipping {y_train[i][0]}'s dataset."
            )
            continue

    return (all_precision, all_recall, all_accuracy), (
        misclassified_images,
        misclassified_true_labels,
        misclassified_pred_labels,
    )

In [ ]:
def eval_eigenface(useSym):
    all_precision = []
    all_recall = []
    all_accuracy = []

    misclassified_images = []
    misclassified_true_labels = []
    misclassified_pred_labels = []

    for i in range(len(data)):
        try:
            training_label_set_person_i = y_train[i]
            training_set_person_i = (
                normalize_list(x_train[i], useSym) if frontalize_face else x_train[i]
            )
            testing_label_set_person_i = y_test[i]
            testing_set_person_i = (
                normalize_list(x_test[i], useSym) if frontalize_face else x_test[i]
            )

            train_images_flat = np.array([p.flatten() for p in training_set_person_i])
            test_images_flat = np.array([p.flatten() for p in testing_set_person_i])

            pca = PCA(n_components=min(5, train_images_flat.shape[0] - 1))
            train_pca = pca.fit_transform(train_images_flat)
            test_pca = pca.transform(test_images_flat)

            knn = KNeighborsClassifier(n_neighbors=1)
            knn.fit(train_pca, training_label_set_person_i)
            test_predictions = knn.predict(test_pca)

            precision, recall, accuracy = calculate_metrics(
                testing_label_set_person_i, test_predictions
            )

            all_precision.append(precision)
            all_recall.append(recall)
            all_accuracy.append(accuracy)

            (
                misclassified_batch_images,
                misclassified_batch_true_labels,
                misclassified_batch_pred_labels,
            ) = track_misclassifications(
                testing_set_person_i, testing_label_set_person_i, test_predictions
            )

            misclassified_images.extend(misclassified_batch_images)
            misclassified_true_labels.extend(misclassified_batch_true_labels)
            misclassified_pred_labels.extend(misclassified_batch_pred_labels)
        except Exception as e:
            print(
                f"An error occurred at index {i} for one of the images: {e} Skipping {y_train[i][0]}'s dataset."
            )
            continue

    return (all_precision, all_recall, all_accuracy), (
        misclassified_images,
        misclassified_true_labels,
        misclassified_pred_labels,
    )

In [ ]:
metrics_fisherface, misclassified_fisherface = eval_fisherface(useSym=False)

In [ ]:
print_metrics("Using FisherFace", *metrics_fisherface)

In [ ]:
visualize_misclassifications("Using FisherFace", *misclassified_fisherface)

In [ ]:
metrics_sift, misclassified_sift = eval_eigenface(useSym=False)

In [ ]:
print_metrics("Using EigenFace", *metrics_sift)

In [ ]:
visualize_misclassifications("Using EigenFace", *misclassified_sift)